## Loading Data Sets

In [1]:
import time
import pandas as pd
import os
import numpy as np
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [2]:
def mergetable(colname, pref, mtable):
    
    dftemp = pd.get_dummies(mtable, columns = [colname], prefix = pref)
    
    collist = list(dftemp.columns[dftemp.columns.str.startswith(pref)])
    
    collist.append('SK_ID_CURR')
    
    dftemp = dftemp.groupby('SK_ID_CURR', as_index = False).sum()[collist]
    
    df = pd.merge(mtable,dftemp,on = 'SK_ID_CURR', how = 'inner')
    
    return df

In [3]:
def callmerge(coldict, mtable):
    
    for i,(pref,colname) in enumerate(coldict.items()):
        
        mtable = mergetable(colname, pref, mtable)

        print("The Tables from iteration # {} are now merged\n" . format(i+1))

        if i == len(coldict) - 1:
            print("All tables are now merged")
            
    return mtable

In [4]:
start_time = time.time()

In [5]:
os.chdir('C:/Users/barun/Desktop/SB/Home Credit Project/')

In [6]:
print(os.getcwd())

C:\Users\barun\Desktop\SB\Home Credit Project


In [7]:
df_train = pd.read_csv("./datasets/application_train.csv")
df0 = df_train.copy()

In [8]:
previous_app = pd.read_csv('./datasets/previous_application.csv')
df2 = previous_app.copy()

In [9]:
bureau = pd.read_csv('./datasets/bureau.csv')
df1 = bureau.copy()

bureau_bal = pd.read_csv('./datasets/bureau_balance.csv')
df11 = bureau_bal.copy()

pos = pd.read_csv('./datasets/POS_CASH_balance.csv')
df21 = pos.copy()

installments = pd.read_csv('./datasets/installments_payments.csv')
df22 = installments.copy()

credit = pd.read_csv('./datasets/credit_card_balance.csv')
df23 = credit.copy()

In [10]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [11]:
df2.head()

SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  \
0     2030495      271877     Consumer loans     1730.430          17145.0   
1     2802425      108129         Cash loans    25188.615         607500.0   
2     2523466      122040         Cash loans    15060.735         112500.0   
3     2819243      176158         Cash loans    47041.335         450000.0   
4     1784265      202054         Cash loans    31924.395         337500.0   

   AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE WEEKDAY_APPR_PROCESS_START  \
0     17145.0               0.0          17145.0                   SATURDAY   
1    679671.0               NaN         607500.0                   THURSDAY   
2    136444.5               NaN         112500.0                    TUESDAY   
3    470790.0               NaN         450000.0                     MONDAY   
4    404055.0               NaN         337500.0                   THURSDAY   

   HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  \
0                       15                           Y   
1                       11                           Y   
2                       11                           Y   
3                        7                           Y   
4                        9                           Y   

   NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  \
0                       1                0.0               0.182832   
1                       1                NaN                    NaN   
2                       1                NaN                    NaN   
3                       1                NaN                    NaN   
4                       1                NaN                    NaN   

   RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  \
0                  0.867336                    XAP             Approved   
1                       NaN                    XNA             Approved   
2                       NaN                    XNA             Approved   
3                       NaN                    XNA             Approved   
4                       NaN                Repairs              Refused   

   DAYS_DECISION      NAME_PAYMENT_TYPE CODE_REJECT_REASON  NAME_TYPE_SUITE  \
0            -73  Cash through the bank                XAP              NaN   
1           -164                    XNA                XAP    Unaccompanied   
2           -301  Cash through the bank                XAP  Spouse, partner   
3           -512  Cash through the bank                XAP              NaN   
4           -781  Cash through the bank                 HC              NaN   

  NAME_CLIENT_TYPE NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE  \
0         Repeater              Mobile            POS               XNA   
1         Repeater                 XNA           Cash            x-sell   
2         Repeater                 XNA           Cash            x-sell   
3         Repeater                 XNA           Cash            x-sell   
4         Repeater                 XNA           Cash           walk-in   

              CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  \
0             Country-wide                35         Connectivity   
1           Contact center                -1                  XNA   
2  Credit and cash offices                -1                  XNA   
3  Credit and cash offices                -1                  XNA   
4  Credit and cash offices                -1                  XNA   

   CNT_PAYMENT NAME_YIELD_GROUP       PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  \
0         12.0           middle  POS mobile with interest            365243.0   
1         36.0       low_action          Cash X-Sell: low            365243.0   
2         12.0             high         Cash X-Sell: high            365243.0   
3         12.0           middle       Cash X-Sell: middle            365243.0   
4         24.0             high         Cash Street: high                 NaN   

   D

In [12]:
df2['WEEKDAY_APPR_PROCESS_START'].value_counts()

TUESDAY      255118
WEDNESDAY    255010
MONDAY       253557
FRIDAY       252048
THURSDAY     249099
SATURDAY     240631
SUNDAY       164751
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64

##  Converting weekday aplication Process start date as a numeric field

In [13]:
df2['WEEKDAY_APPR_PROCESS_START'] = df2['WEEKDAY_APPR_PROCESS_START'].map({'SUNDAY':0,'MONDAY': 1, 'TUESDAY':2, 'WEDNESDAY':3,'THURSDAY':4,
                                      'FRIDAY': 5, 'SATURDAY' : 6})

In [14]:
df2['WEEKDAY_APPR_PROCESS_START'].head(10)

0    6
1    4
2    2
3    1
4    4
5    6
6    2
7    1
8    1
9    6
Name: WEEKDAY_APPR_PROCESS_START, dtype: int64

## Feature Engineering on previous application data set to get the counts of different categories

In [15]:
df2['NAME_CONTRACT_STATUS'].value_counts()

Approved        1036781
Canceled         316319
Refused          290678
Unused offer      26436
Name: NAME_CONTRACT_STATUS, dtype: int64

In [16]:
df2['NAME_CONTRACT_TYPE'].value_counts()

Cash loans         747553
Consumer loans     729151
Revolving loans    193164
XNA                   346
Name: NAME_CONTRACT_TYPE, dtype: int64

In [17]:
df2['NAME_CASH_LOAN_PURPOSE'].value_counts()

XAP                                 922661
XNA                                 677918
Repairs                              23765
Other                                15608
Urgent needs                          8412
Buying a used car                     2888
Building a house or an annex          2693
Everyday expenses                     2416
Medicine                              2174
Payments on other loans               1931
Education                             1573
Journey                               1239
Purchase of electronic equipment      1061
Buying a new car                      1012
Wedding / gift / holiday               962
Buying a home                          865
Car repairs                            797
Furniture                              749
Buying a holiday home / land           533
Business development                   426
Gasification / water supply            300
Buying a garage                        136
Hobby                                   55
Money for a

In [18]:
df2['NAME_GOODS_CATEGORY'].value_counts()

XNA                         950809
Mobile                      224708
Consumer Electronics        121576
Computers                   105769
Audio/Video                  99441
Furniture                    53656
Photo / Cinema Equipment     25021
Construction Materials       24995
Clothing and Accessories     23554
Auto Accessories              7381
Jewelry                       6290
Homewares                     5023
Medical Supplies              3843
Vehicles                      3370
Sport and Leisure             2981
Gardening                     2668
Other                         2554
Office Appliances             2333
Tourism                       1659
Medicine                      1550
Direct Sales                   446
Fitness                        209
Additional Service             128
Education                      107
Weapon                          77
Insurance                       64
Animals                          1
House Construction               1
Name: NAME_GOODS_CAT

In [19]:
df2['NAME_CLIENT_TYPE'].value_counts()

Repeater     1231261
New           301363
Refreshed     135649
XNA             1941
Name: NAME_CLIENT_TYPE, dtype: int64

In [20]:
df2['CHANNEL_TYPE'].value_counts()

Credit and cash offices       719968
Country-wide                  494690
Stone                         212083
Regional / Local              108528
Contact center                 71297
AP+ (Cash loan)                57046
Channel of corporate sales      6150
Car dealer                       452
Name: CHANNEL_TYPE, dtype: int64

In [21]:
df2['NAME_TYPE_SUITE'].value_counts()

Unaccompanied      508970
Family             213263
Spouse, partner     67069
Children            31566
Other_B             17624
Other_A              9077
Group of people      2240
Name: NAME_TYPE_SUITE, dtype: int64

In [22]:
df2['NAME_PORTFOLIO'].value_counts()

POS      691011
Cash     461563
XNA      372230
Cards    144985
Cars        425
Name: NAME_PORTFOLIO, dtype: int64

In [23]:
df2['NAME_PRODUCT_TYPE'].value_counts()

XNA        1063666
x-sell      456287
walk-in     150261
Name: NAME_PRODUCT_TYPE, dtype: int64

In [24]:
df2['NAME_SELLER_INDUSTRY'].value_counts()

XNA                     855720
Consumer electronics    398265
Connectivity            276029
Furniture                57849
Construction             29781
Clothing                 23949
Industry                 19194
Auto technology           4990
Jewelry                   2709
MLM partners              1215
Tourism                    513
Name: NAME_SELLER_INDUSTRY, dtype: int64

In [25]:
coldict = {'PREV_NCS_COUNT' : 'NAME_CONTRACT_STATUS',
           'PREV_NCLT_COUNT' : 'NAME_CLIENT_TYPE',
           'PREV_NCT_COUNT' : 'NAME_CONTRACT_TYPE',
           'PREV_NCLP_COUNT' : 'NAME_CASH_LOAN_PURPOSE',
           'PREV_NTS_COUNT' : 'NAME_TYPE_SUITE',
           'PREV_NGC_COUNT' : 'NAME_GOODS_CATEGORY',
           'PREV_NPF_COUNT' : 'NAME_PORTFOLIO',
           'PREV_CT_COUNT' :  'CHANNEL_TYPE',
           'PREV_NPT_COUNT' : 'NAME_PRODUCT_TYPE',
           'PREV_NSI_COUNT' : 'NAME_SELLER_INDUSTRY'}

In [26]:
df2.shape

(1670214, 37)

In [27]:
df2 = callmerge(coldict,df2)

The Tables from iteration # 1 are now merged

The Tables from iteration # 2 are now merged

The Tables from iteration # 3 are now merged

The Tables from iteration # 4 are now merged

The Tables from iteration # 5 are now merged

The Tables from iteration # 6 are now merged

The Tables from iteration # 7 are now merged

The Tables from iteration # 8 are now merged

The Tables from iteration # 9 are now merged

The Tables from iteration # 10 are now merged

All tables are now merged


In [28]:
df2.shape

(1670214, 136)

In [29]:
df2.head()

SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  \
0     2030495      271877     Consumer loans     1730.430          17145.0   
1     1696966      271877     Consumer loans    68258.655        1800000.0   
2     2154916      271877     Consumer loans    12417.390         108400.5   
3     2802425      108129         Cash loans    25188.615         607500.0   
4     1536272      108129         Cash loans    21709.125         450000.0   

   AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  WEEKDAY_APPR_PROCESS_START  \
0     17145.0               0.0          17145.0                           6   
1   1754721.0          180000.0        1800000.0                           6   
2    119848.5               0.0         108400.5                           0   
3    679671.0               NaN         607500.0                           4   
4    512370.0               NaN         450000.0                           3   

   HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  \
0                       15                           Y   
1                       18                           Y   
2                       14                           Y   
3                       11                           Y   
4                        9                           Y   

   NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  \
0                       1           0.000000               0.182832   
1                       1           0.101325                    NaN   
2                       1           0.000000                    NaN   
3                       1                NaN                    NaN   
4                       1                NaN                    NaN   

   RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  \
0                  0.867336                    XAP             Approved   
1                       NaN                    XAP              Refused   
2                       NaN                    XAP             Approved   
3                       NaN                    XNA             Approved   
4                       NaN                    XNA             Approved   

   DAYS_DECISION      NAME_PAYMENT_TYPE CODE_REJECT_REASON NAME_TYPE_SUITE  \
0            -73  Cash through the bank                XAP             NaN   
1           -472  Cash through the bank                SCO             NaN   
2           -548  Cash through the bank                XAP             NaN   
3           -164                    XNA                XAP   Unaccompanied   
4           -515                    XNA                XAP             NaN   

  NAME_CLIENT_TYPE       NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE  \
0         Repeater                    Mobile            POS               XNA   
1         Repeater  Clothing and Accessories            POS               XNA   
2              New                 Furniture            POS               XNA   
3         Repeater                       XNA           Cash            x-sell   
4         Repeater                       XNA           Cash            x-sell   

       CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  CNT_PAYMENT  \
0      Country-wide                35         Connectivity         12.0   
1  Regional / Local                55            Furniture         36.0   
2             Stone               196            Furniture         12.0   
3    Contact center                -1                  XNA         36.0   
4   AP+ (Cash loan)                 6                  XNA         36.0   

  NAME_YIELD_GROUP         PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  \
0           middle    POS mobile with interest            365243.0   
1       low_normal  POS industry with interest                 NaN   
2           middle  POS industry with interest            365243.0   
3       low_action            Cash X-Sell: low            365243.0   
4       low_normal            Cash X-Sell: low            365243.0   

In [30]:
dftmp = df2.groupby('SK_ID_CURR', as_index = False).count()[['SK_ID_CURR','SK_ID_PREV']]
dftmp = dftmp.rename(columns = {'SK_ID_PREV' : 'PREV_LOAN_COUNT'})
dfplc = pd.merge(df2, dftmp, on = 'SK_ID_CURR', how = 'inner')

In [31]:
dfplc.shape

(1670214, 137)

In [32]:
dfplc.head()

SK_ID_PREV  SK_ID_CURR NAME_CONTRACT_TYPE  AMT_ANNUITY  AMT_APPLICATION  \
0     2030495      271877     Consumer loans     1730.430          17145.0   
1     1696966      271877     Consumer loans    68258.655        1800000.0   
2     2154916      271877     Consumer loans    12417.390         108400.5   
3     2802425      108129         Cash loans    25188.615         607500.0   
4     1536272      108129         Cash loans    21709.125         450000.0   

   AMT_CREDIT  AMT_DOWN_PAYMENT  AMT_GOODS_PRICE  WEEKDAY_APPR_PROCESS_START  \
0     17145.0               0.0          17145.0                           6   
1   1754721.0          180000.0        1800000.0                           6   
2    119848.5               0.0         108400.5                           0   
3    679671.0               NaN         607500.0                           4   
4    512370.0               NaN         450000.0                           3   

   HOUR_APPR_PROCESS_START FLAG_LAST_APPL_PER_CONTRACT  \
0                       15                           Y   
1                       18                           Y   
2                       14                           Y   
3                       11                           Y   
4                        9                           Y   

   NFLAG_LAST_APPL_IN_DAY  RATE_DOWN_PAYMENT  RATE_INTEREST_PRIMARY  \
0                       1           0.000000               0.182832   
1                       1           0.101325                    NaN   
2                       1           0.000000                    NaN   
3                       1                NaN                    NaN   
4                       1                NaN                    NaN   

   RATE_INTEREST_PRIVILEGED NAME_CASH_LOAN_PURPOSE NAME_CONTRACT_STATUS  \
0                  0.867336                    XAP             Approved   
1                       NaN                    XAP              Refused   
2                       NaN                    XAP             Approved   
3                       NaN                    XNA             Approved   
4                       NaN                    XNA             Approved   

   DAYS_DECISION      NAME_PAYMENT_TYPE CODE_REJECT_REASON NAME_TYPE_SUITE  \
0            -73  Cash through the bank                XAP             NaN   
1           -472  Cash through the bank                SCO             NaN   
2           -548  Cash through the bank                XAP             NaN   
3           -164                    XNA                XAP   Unaccompanied   
4           -515                    XNA                XAP             NaN   

  NAME_CLIENT_TYPE       NAME_GOODS_CATEGORY NAME_PORTFOLIO NAME_PRODUCT_TYPE  \
0         Repeater                    Mobile            POS               XNA   
1         Repeater  Clothing and Accessories            POS               XNA   
2              New                 Furniture            POS               XNA   
3         Repeater                       XNA           Cash            x-sell   
4         Repeater                       XNA           Cash            x-sell   

       CHANNEL_TYPE  SELLERPLACE_AREA NAME_SELLER_INDUSTRY  CNT_PAYMENT  \
0      Country-wide                35         Connectivity         12.0   
1  Regional / Local                55            Furniture         36.0   
2             Stone               196            Furniture         12.0   
3    Contact center                -1                  XNA         36.0   
4   AP+ (Cash loan)                 6                  XNA         36.0   

  NAME_YIELD_GROUP         PRODUCT_COMBINATION  DAYS_FIRST_DRAWING  \
0           middle    POS mobile with interest            365243.0   
1       low_normal  POS industry with interest                 NaN   
2           middle  POS industry with interest            365243.0   
3       low_action            Cash X-Sell: low            365243.0   
4       low_normal            Cash X-Sell: low            365243.0   

In [33]:
# Categories of the column NAME_CONTRACT_STATUS
COL_PREV_CS = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NCS')])

# Categories of the column NAME_CLIENT_TYPE
COL_PREV_CLT = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NCLT')])

# Categories of the column NAME_CONTRACT_TYPE
COL_PREV_CONT = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NCT')])

# Categories of the column NAME_CASH_LOAN_PURPOSE
COL_PREV_CLP = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NCLP')])

# Categories of the column NAME_TYPE_SUITE
COL_PREV_NTS = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NTS')])


# Categories of the column NAME_GOODS_CATEGORY
COL_PREV_NGC = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NGC')])

# Categories of the column NAME_PORTFOLIO
COL_PREV_PFL = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NPF')])

# Categories of the column CHANNEL_TYPE
COL_PREV_CHNL = list(dfplc.columns[dfplc.columns.str.startswith('PREV_CT')])

# Categories of the column 'NAME_PRODUCT_TYPE'
COL_PREV_NPT = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NPT')])

# count of previous loans by categories of the column 'NAME_SELLER_INDUSTRY'
COL_PREV_NSI = list(dfplc.columns[dfplc.columns.str.startswith('PREV_NSI')])

# count of previous loans taken by the applicant 
COL_PREV_LOAN = list(dfplc.columns[dfplc.columns.str.startswith('PREV_LOAN')])


In [34]:
noncountfields = ['SK_ID_CURR','AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','RATE_INTEREST_PRIMARY' ]
grpcollist = noncountfields + COL_PREV_CS + COL_PREV_CLT + COL_PREV_CONT + COL_PREV_CLP + COL_PREV_NTS + \
COL_PREV_NGC + COL_PREV_PFL + COL_PREV_CHNL + COL_PREV_NPT + COL_PREV_NSI + COL_PREV_LOAN 

## After Feature Engineering Previous application data set Grouped the mean counts by SK_ID_CURR

In [35]:
# taking the mean of each of the counts calculated previously, group by each current applicant id
dfinal2 = dfplc.groupby('SK_ID_CURR', as_index = False).mean()[grpcollist]

In [36]:
dfinal2['WEEKDAY_APPR_PROCESS_START'] = round(dfinal2['WEEKDAY_APPR_PROCESS_START'])

In [37]:
dfinal2.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    NaN                        1   
1                 9.000000                    NaN                        1   
2                14.666667                    NaN                        3   
3                 5.000000                    NaN                        1   
4                10.500000                    NaN                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [38]:
df1.head()

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0      215354       5714462        Closed      currency 1         -497   
1      215354       5714463        Active      currency 1         -208   
2      215354       5714464        Active      currency 1         -203   
3      215354       5714465        Active      currency 1         -203   
4      215354       5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                     NaN                   0        225000.0   
2                     NaN                   0        464323.5   
3                     NaN                   0         90000.0   
4                 77674.5                   0       2700000.0   

   AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
0                  0.0                   NaN                     0.0   
1             171342.0                   NaN                     0.0   
2                  NaN                   NaN                     0.0   
3                  NaN                   NaN                     0.0   
4                  NaN                   NaN                     0.0   

       CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  
0  Consumer credit                -131          NaN  
1      Credit card                 -20          NaN  
2  Consumer credit                 -16          NaN  
3      Credit card                 -16          NaN  
4  Consumer credit                 -21          NaN

# Creating New Features from pos, credit and installment files

In [39]:
pos.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
0     1803195      182943             -31            48.0   
1     1715348      367990             -33            36.0   
2     1784872      397406             -32            12.0   
3     1903291      269225             -35            48.0   
4     2341044      334279             -35            36.0   

   CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
0                   45.0               Active       0           0  
1                   35.0               Active       0           0  
2                    9.0               Active       0           0  
3                   42.0               Active       0           0  
4                   35.0               Active       0           0

In [40]:
postemp = pos.groupby('SK_ID_CURR',as_index = False).agg({'SK_ID_PREV': 'nunique','CNT_INSTALMENT':'mean','SK_DPD':[lambda x : np.count_nonzero(x),'max','mean']})
postemp.columns = ['SK_ID_CURR','POS_SK_ID_PREV_COUNT','POS_CREDIT_TERM_MEAN','POS_DUE_DAYS_COUNT','POS_DUE_DAYS_MAX','POS_DUE_DAYS_MEAN']

postemp.head()

SK_ID_CURR  POS_SK_ID_PREV_COUNT  POS_CREDIT_TERM_MEAN  POS_DUE_DAYS_COUNT  \
0      100001                     2              4.000000                   1   
1      100002                     1             24.000000                   0   
2      100003                     3             10.107143                   0   
3      100004                     1              3.750000                   0   
4      100005                     1             11.700000                   0   

   POS_DUE_DAYS_MAX  POS_DUE_DAYS_MEAN  
0                 7           0.777778  
1                 0           0.000000  
2                 0           0.000000  
3                 0           0.000000  
4                 0           0.000000

In [41]:
credit.head()

credit['NAME_CONTRACT_STATUS'] = credit['NAME_CONTRACT_STATUS'].astype('category')
credit['NAME_CONTRACT_STATUS'].dtype

CategoricalDtype(categories=['Active', 'Approved', 'Completed', 'Demand', 'Refused',
                  'Sent proposal', 'Signed'],
                 ordered=False)

In [42]:
credit['exceedslimitby'] = credit['AMT_BALANCE'] - credit['AMT_CREDIT_LIMIT_ACTUAL']
credit.loc[credit['exceedslimitby'] >= 0,'exceedsflag'] = 1
credit.loc[credit['exceedslimitby'] < 0,'exceedsflag'] = 0

In [43]:
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_BALANCE'] >= 0,'fullpaymentflag'] = 1
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_BALANCE'] < 0,'fullpaymentflag'] = 0

In [44]:
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_INST_MIN_REGULARITY'] >= 0,'minpaymentflag'] = 1
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_INST_MIN_REGULARITY'] < 0,'minpaymentflag'] = 0

In [45]:
crtemp1 = credit.groupby('SK_ID_CURR', as_index = False).agg({'AMT_BALANCE':'mean','AMT_CREDIT_LIMIT_ACTUAL':'mean',
'exceedslimitby':'mean','exceedsflag':'mean','fullpaymentflag':'mean','minpaymentflag':'mean'
,'AMT_DRAWINGS_CURRENT':'mean','CNT_DRAWINGS_CURRENT':'mean'})

In [46]:
crtemp1.head()

SK_ID_CURR   AMT_BALANCE  AMT_CREDIT_LIMIT_ACTUAL  exceedslimitby  \
0      100006      0.000000            270000.000000  -270000.000000   
1      100011  54482.111149            164189.189189  -109707.078041   
2      100013  18159.919219            131718.750000  -113558.830781   
3      100021      0.000000            675000.000000  -675000.000000   
4      100023      0.000000            135000.000000  -135000.000000   

   exceedsflag  fullpaymentflag  minpaymentflag  AMT_DRAWINGS_CURRENT  \
0     0.000000         1.000000        1.000000              0.000000   
1     0.040541         0.567568        0.986301           2432.432432   
2     0.010417         0.822917        0.921348           5953.125000   
3     0.000000         1.000000        1.000000              0.000000   
4     0.000000         1.000000        1.000000              0.000000   

   CNT_DRAWINGS_CURRENT  
0              0.000000  
1              0.054054  
2              0.239583  
3              0.000000  
4              0.000000

In [47]:
crtemp = pd.get_dummies(credit, columns = ['NAME_CONTRACT_STATUS'], prefix = 'CREDIT_CONT_STAT')
collist = list(crtemp.columns[crtemp.columns.str.startswith('CREDIT_CONT_STAT')])
collist.append('SK_ID_CURR')
collist.append('SK_ID_PREV')

In [48]:
credit['NAME_CONTRACT_STATUS'].value_counts()

crtemp = crtemp.groupby(['SK_ID_CURR','SK_ID_PREV'], as_index = False).max()[collist]

crtemp.head()

CREDIT_CONT_STAT_Active  CREDIT_CONT_STAT_Approved  \
0                        1                          0   
1                        1                          0   
2                        1                          0   
3                        1                          0   
4                        1                          0   

   CREDIT_CONT_STAT_Completed  CREDIT_CONT_STAT_Demand  \
0                           0                        0   
1                           0                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   CREDIT_CONT_STAT_Refused  CREDIT_CONT_STAT_Sent proposal  \
0                         0                               0   
1                         0                               0   
2                         0                               0   
3                         0                               0   
4                         0                               0   

   CREDIT_CONT_STAT_Signed  SK_ID_CURR  SK_ID_PREV  
0                        0      100006     1489396  
1                        0      100011     1843384  
2                        0      100013     2038692  
3                        0      100021     2594025  
4                        0      100023     1499902

In [49]:
crtemp2 = crtemp.groupby('SK_ID_CURR', as_index = False).sum()
crtemp2.drop(columns = 'SK_ID_PREV', inplace = True )
crtemp2.head()

SK_ID_CURR  CREDIT_CONT_STAT_Active  CREDIT_CONT_STAT_Approved  \
0      100006                        1                          0   
1      100011                        1                          0   
2      100013                        1                          0   
3      100021                        1                          0   
4      100023                        1                          0   

   CREDIT_CONT_STAT_Completed  CREDIT_CONT_STAT_Demand  \
0                           0                        0   
1                           0                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   CREDIT_CONT_STAT_Refused  CREDIT_CONT_STAT_Sent proposal  \
0                         0                               0   
1                         0                               0   
2                         0                               0   
3                         0                               0   
4                         0                               0   

   CREDIT_CONT_STAT_Signed  
0                        0  
1                        0  
2                        0  
3                        0  
4                        0

In [50]:
credittemp = pd.merge(crtemp1,crtemp2, on = 'SK_ID_CURR', how = 'left')

In [51]:
credittemp.head()

credittemp.shape

(103558, 16)

In [52]:
credittemp.columns = ['SK_ID_CURR','CR_AMT_BALANCE_MEAN','CR_CREDIT_LIMIT_MEAN','CR_EXCEED_LIMIT_AMT_MEAN',
                      'CR_EXCEED_LIMIT_CNT_MEAN','CR_FULL_PAYMENT_COUNT_MEAN','CR_MIN_PAYMENT_COUNT_MEAN',
                     'CR_DRAWINGS_AMT_MEAN','CR_DRAWINGS_CNT_MEAN','CR_CREDIT_CNT_ACTIVE','CR_CREDIT_CNT_APPROVED',
'CR_CREDIT_CNT_COMPLETED','CR_CREDIT_CNT_DEMAND','CR_CREDIT_CNT_REFUSED','CR_CREDIT_CNT_SENTPROPOSAL','CR_CREDIT_CNT_SIGNED']

credittemp.head()

SK_ID_CURR  CR_AMT_BALANCE_MEAN  CR_CREDIT_LIMIT_MEAN  \
0      100006             0.000000         270000.000000   
1      100011         54482.111149         164189.189189   
2      100013         18159.919219         131718.750000   
3      100021             0.000000         675000.000000   
4      100023             0.000000         135000.000000   

   CR_EXCEED_LIMIT_AMT_MEAN  CR_EXCEED_LIMIT_CNT_MEAN  \
0            -270000.000000                  0.000000   
1            -109707.078041                  0.040541   
2            -113558.830781                  0.010417   
3            -675000.000000                  0.000000   
4            -135000.000000                  0.000000   

   CR_FULL_PAYMENT_COUNT_MEAN  CR_MIN_PAYMENT_COUNT_MEAN  \
0                    1.000000                   1.000000   
1                    0.567568                   0.986301   
2                    0.822917                   0.921348   
3                    1.000000                   1.000000   
4                    1.000000                   1.000000   

   CR_DRAWINGS_AMT_MEAN  CR_DRAWINGS_CNT_MEAN  CR_CREDIT_CNT_ACTIVE  \
0              0.000000              0.000000                     1   
1           2432.432432              0.054054                     1   
2           5953.125000              0.239583                     1   
3              0.000000              0.000000                     1   
4              0.000000              0.000000                     1   

   CR_CREDIT_CNT_APPROVED  CR_CREDIT_CNT_COMPLETED  CR_CREDIT_CNT_DEMAND  \
0                       0                        0                     0   
1                       0                        0                     0   
2                       0                        0                     0   
3                       0                        1                     0   
4                       0                        0                     0   

   CR_CREDIT_CNT_REFUSED  CR_CREDIT_CNT_SENTPROPOSAL  CR_CREDIT_CNT_SIGNED  
0                      0                           0                     0  
1                      0                           0                     0  
2                      0                           0                     0  
3                      0                           0                     0  
4                      0                           0                     0

In [53]:
installments.head()

SK_ID_PREV  SK_ID_CURR  NUM_INSTALMENT_VERSION  NUM_INSTALMENT_NUMBER  \
0     1054186      161674                     1.0                      6   
1     1330831      151639                     0.0                     34   
2     2085231      193053                     2.0                      1   
3     2452527      199697                     1.0                      3   
4     2714724      167756                     1.0                      2   

   DAYS_INSTALMENT  DAYS_ENTRY_PAYMENT  AMT_INSTALMENT  AMT_PAYMENT  
0          -1180.0             -1187.0        6948.360     6948.360  
1          -2156.0             -2156.0        1716.525     1716.525  
2            -63.0               -63.0       25425.000    25425.000  
3          -2418.0             -2426.0       24350.130    24350.130  
4          -1383.0             -1366.0        2165.040     2160.585

In [54]:
installments['daysoverdue'] = abs(installments['DAYS_ENTRY_PAYMENT'])-abs(installments['DAYS_INSTALMENT'])

instemp = installments.groupby('SK_ID_CURR', as_index = False).agg({'SK_ID_PREV':'nunique','NUM_INSTALMENT_NUMBER':'max',
                                                         'AMT_INSTALMENT':'sum','daysoverdue':['mean','max']})


instemp.columns = ['SK_ID_CURR','INS_SK_ID_PREV_COUNT','INS_NUM_MAX','INS_AMT_SUM','INS_DAYS_OVERDUE_MEAN','INS_DAYS_OVERDUE_MAX']

instemp.head()

SK_ID_CURR  INS_SK_ID_PREV_COUNT  INS_NUM_MAX  INS_AMT_SUM  \
0      100001                     2            4    41195.925   
1      100002                     1           19   219625.695   
2      100003                     3           12  1618864.650   
3      100004                     1            3    21288.465   
4      100005                     1            9    56161.845   

   INS_DAYS_OVERDUE_MEAN  INS_DAYS_OVERDUE_MAX  
0               7.285714                  36.0  
1              20.421053                  31.0  
2               7.160000                  14.0  
3               7.666667                  11.0  
4              23.555556                  37.0

In [55]:
### Merging the engineered pos, installment and credit data set with engineered previous application data set to build the modified previous application data set included with all the newly derived features 

prevtemp = dfinal2

prevtemp.head()

prevtemp.shape

prevtemp1 = pd.merge(prevtemp, credittemp, on = 'SK_ID_CURR', how = 'left')
prevtemp1.fillna(0, inplace = True)
prevtemp2 = pd.merge(prevtemp1, instemp, on = 'SK_ID_CURR', how = 'left')
prevtempfinal = pd.merge(prevtemp2,postemp,on = 'SK_ID_CURR', how = 'left')
#prevtempfinal.drop(columns = ['SK_ID_PREV_COUNT_x'], axis =0 , inplace = True)
prevtempfinal.fillna(0, inplace = True)

In [56]:
prevtempfinal.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [57]:
prevtempfinal.shape

(338857, 130)

In [58]:
prevtempfinal.columns

Index(['SK_ID_CURR', 'AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'RATE_INTEREST_PRIMARY',
       'PREV_NCS_COUNT_Approved', 'PREV_NCS_COUNT_Canceled',
       'PREV_NCS_COUNT_Refused', 'PREV_NCS_COUNT_Unused offer',
       'PREV_NCLT_COUNT_New',
       ...
       'INS_SK_ID_PREV_COUNT', 'INS_NUM_MAX', 'INS_AMT_SUM',
       'INS_DAYS_OVERDUE_MEAN', 'INS_DAYS_OVERDUE_MAX', 'POS_SK_ID_PREV_COUNT',
       'POS_CREDIT_TERM_MEAN', 'POS_DUE_DAYS_COUNT', 'POS_DUE_DAYS_MAX',
       'POS_DUE_DAYS_MEAN'],
      dtype='object', length=130)

In [59]:
dfinal2 = prevtempfinal

dfinal2.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [60]:
dfinal2.shape

(338857, 130)

## Finding the MAX(STATUS) of each SK_ID_CURR from the bureau_balance data set  and then joining it with the bureau data set

In [62]:
df11['STATUS'] = df11['STATUS'].replace('C', '-2')
df11['STATUS'] = df11['STATUS'].replace('X', '-1')
df11['STATUS'] = df11['STATUS'].astype(int)

In [63]:
df111 = df11.groupby('SK_ID_BUREAU', as_index = False).max()[['SK_ID_BUREAU','STATUS']]

In [64]:
df111.head()

SK_ID_BUREAU  STATUS
0       5001709      -1
1       5001710       0
2       5001711       0
3       5001712       0
4       5001713      -1

In [65]:
dfl = pd.merge(df1, df111, on = 'SK_ID_BUREAU', how = 'left')

In [66]:
dfl.shape

(1716428, 18)

## Feature Engineering on the above joined bureau data set to get the counts of bureau credits by their current status (closed/active etc.) 

In [68]:
dfl.CREDIT_ACTIVE.value_counts()

Closed      1079273
Active       630607
Sold           6527
Bad debt         21
Name: CREDIT_ACTIVE, dtype: int64

In [69]:
bureaucoldict = {'CREDIT_ACTIVE_STAT':'CREDIT_ACTIVE'}

In [70]:
dfl = callmerge(bureaucoldict,dfl)

The Tables from iteration # 1 are now merged

All tables are now merged


In [71]:
dfl.head()

SK_ID_CURR  SK_ID_BUREAU CREDIT_ACTIVE CREDIT_CURRENCY  DAYS_CREDIT  \
0      215354       5714462        Closed      currency 1         -497   
1      215354       5714463        Active      currency 1         -208   
2      215354       5714464        Active      currency 1         -203   
3      215354       5714465        Active      currency 1         -203   
4      215354       5714466        Active      currency 1         -629   

   CREDIT_DAY_OVERDUE  DAYS_CREDIT_ENDDATE  DAYS_ENDDATE_FACT  \
0                   0               -153.0             -153.0   
1                   0               1075.0                NaN   
2                   0                528.0                NaN   
3                   0                  NaN                NaN   
4                   0               1197.0                NaN   

   AMT_CREDIT_MAX_OVERDUE  CNT_CREDIT_PROLONG  AMT_CREDIT_SUM  \
0                     NaN                   0         91323.0   
1                     NaN                   0        225000.0   
2                     NaN                   0        464323.5   
3                     NaN                   0         90000.0   
4                 77674.5                   0       2700000.0   

   AMT_CREDIT_SUM_DEBT  AMT_CREDIT_SUM_LIMIT  AMT_CREDIT_SUM_OVERDUE  \
0                  0.0                   NaN                     0.0   
1             171342.0                   NaN                     0.0   
2                  NaN                   NaN                     0.0   
3                  NaN                   NaN                     0.0   
4                  NaN                   NaN                     0.0   

       CREDIT_TYPE  DAYS_CREDIT_UPDATE  AMT_ANNUITY  STATUS  \
0  Consumer credit                -131          NaN     NaN   
1      Credit card                 -20          NaN     NaN   
2  Consumer credit                 -16          NaN     NaN   
3      Credit card                 -16          NaN     NaN   
4  Consumer credit                 -21          NaN     NaN   

   CREDIT_ACTIVE_STAT_Active  CREDIT_ACTIVE_STAT_Bad debt  \
0                          6                            0   
1                          6                            0   
2                          6                            0   
3                          6                            0   
4                          6                            0   

   CREDIT_ACTIVE_STAT_Closed  CREDIT_ACTIVE_STAT_Sold  
0                          5                        0  
1                          5                        0  
2                          5                        0  
3                          5                        0  
4                          5                        0

In [72]:
dfl.columns

Index(['SK_ID_CURR', 'SK_ID_BUREAU', 'CREDIT_ACTIVE', 'CREDIT_CURRENCY',
       'DAYS_CREDIT', 'CREDIT_DAY_OVERDUE', 'DAYS_CREDIT_ENDDATE',
       'DAYS_ENDDATE_FACT', 'AMT_CREDIT_MAX_OVERDUE', 'CNT_CREDIT_PROLONG',
       'AMT_CREDIT_SUM', 'AMT_CREDIT_SUM_DEBT', 'AMT_CREDIT_SUM_LIMIT',
       'AMT_CREDIT_SUM_OVERDUE', 'CREDIT_TYPE', 'DAYS_CREDIT_UPDATE',
       'AMT_ANNUITY', 'STATUS', 'CREDIT_ACTIVE_STAT_Active',
       'CREDIT_ACTIVE_STAT_Bad debt', 'CREDIT_ACTIVE_STAT_Closed',
       'CREDIT_ACTIVE_STAT_Sold'],
      dtype='object')

## After Feature Engineering bureau data set, Group the mean of counts and max of amount fields by SK_ID_CURR

In [73]:
dff = dfl.groupby('SK_ID_CURR', as_index = False).agg({'AMT_CREDIT_MAX_OVERDUE':'max', 'AMT_CREDIT_SUM':'mean',
                                               'AMT_CREDIT_SUM_DEBT':'max', 'CREDIT_ACTIVE_STAT_Active': 'mean', 
                                                'CREDIT_ACTIVE_STAT_Closed':'mean','CREDIT_ACTIVE_STAT_Sold':'mean', 'CREDIT_ACTIVE_STAT_Bad debt':'mean',
                                                      'STATUS' : 'max'})

In [74]:
dff = dff.rename(columns = {'AMT_CREDIT_MAX_OVERDUE':'BRU_AMT_MAX_OVERDUE', 'AMT_CREDIT_SUM' : 'BRU_MEAN_AMT_CREDIT_SUM',
                     'AMT_CREDIT_SUM_DEBT': 'BRU_MAX_CRDT_SUM_DEBT', 'ACTIVE_COUNT' : 'BRU_MEAN_ACTIVE_COUNT',
                     'CLOSED_COUNT' : 'BRU_MEAN_CLOSED_COUNT', 'SOLD_COUNT':'BRU_MEAN_SOLD_COUNT',
                     'BAD_COUNT': 'BRU_MEAN_BAD_COUNT', 'STATUS' : 'BRU_MAX_STATUS'})

In [75]:
dff.BRU_MAX_CRDT_SUM_DEBT.fillna(0, inplace = True)
dff.BRU_AMT_MAX_OVERDUE.fillna(0, inplace = True)
dff.BRU_MEAN_AMT_CREDIT_SUM.fillna(0, inplace = True)
dfl['STATUS'].fillna(-5, inplace = True)

# Merging updated bureau data set with app_train dataset 

In [76]:
dfinal1 = pd.merge(df0, dff, on = 'SK_ID_CURR', how = 'left')

In [77]:
dfinal1.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT  AMT_ANNUITY  \
0               Y             0          202500.0    406597.5      24700.5   
1               N             0          270000.0   1293502.5      35698.5   
2               Y             0           67500.0    135000.0       6750.0   
3               Y             0          135000.0    312682.5      29686.5   
4               Y             0          121500.0    513000.0      21865.5   

   AMT_GOODS_PRICE NAME_TYPE_SUITE NAME_INCOME_TYPE  \
0         351000.0   Unaccompanied          Working   
1        1129500.0          Family    State servant   
2         135000.0   Unaccompanied          Working   
3         297000.0   Unaccompanied          Working   
4         513000.0   Unaccompanied          Working   

             NAME_EDUCATION_TYPE    NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
0  Secondary / secondary special  Single / not married  House / apartment   
1               Higher education               Married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3  Secondary / secondary special        Civil marriage  House / apartment   
4  Secondary / secondary special  Single / not married  House / apartment   

   REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  \
0                    0.018801       -9461           -637            -3648.0   
1                    0.003541      -16765          -1188            -1186.0   
2                    0.010032      -19046           -225            -4260.0   
3                    0.008019      -19005          -3039            -9833.0   
4                    0.028663      -19932          -3038            -4311.0   

   DAYS_ID_PUBLISH  OWN_CAR_AGE  FLAG_MOBIL  FLAG_EMP_PHONE  FLAG_WORK_PHONE  \
0            -2120          NaN           1               1                0   
1             -291          NaN           1               1                0   
2            -2531         26.0           1               1                1   
3            -2437          NaN           1               1                0   
4            -3458          NaN           1               1                0   

   FLAG_CONT_MOBILE  FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  CNT_FAM_MEMBERS  \
0                 1           1           0        Laborers              1.0   
1                 1           1           0      Core staff              2.0   
2                 1           1           0        Laborers              1.0   
3                 1           0           0        Laborers              2.0   
4                 1           0           0      Core staff              1.0   

   REGION_RATING_CLIENT  REGION_RATING_CLIENT_W_CITY  \
0                     2                            2   
1                     1                            1   
2                     2                            2   
3                     2                            2   
4                     2                            2   

  WEEKDAY_APPR_PROCESS_START  HOUR_APPR_PROCESS_START  \
0                  WEDNESDAY                       10   
1                     MONDAY                       11   
2                     MONDAY                        9   
3                  WEDNESDAY                       17   
4                   THURSDAY                       11   

   REG_REGION_NOT_LIVE_REGION  REG_REGION_NOT_WORK_REGION  \
0                           0                           0   
1                           0                           0   
2                        

In [78]:
dfinal2.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

   ## Feature engineering on the installment, pos and credit data sets

In [79]:
pos.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  CNT_INSTALMENT  \
0     1803195      182943             -31            48.0   
1     1715348      367990             -33            36.0   
2     1784872      397406             -32            12.0   
3     1903291      269225             -35            48.0   
4     2341044      334279             -35            36.0   

   CNT_INSTALMENT_FUTURE NAME_CONTRACT_STATUS  SK_DPD  SK_DPD_DEF  
0                   45.0               Active       0           0  
1                   35.0               Active       0           0  
2                    9.0               Active       0           0  
3                   42.0               Active       0           0  
4                   35.0               Active       0           0

In [80]:
postemp = pos.groupby('SK_ID_CURR',as_index = False).agg({'SK_ID_PREV': 'nunique','CNT_INSTALMENT':'mean','SK_DPD':[lambda x : np.count_nonzero(x),'max','mean']})
postemp.columns = ['SK_ID_CURR','POS_SK_ID_PREV_COUNT','POS_CREDIT_TERM_MEAN','POS_DUE_DAYS_COUNT','POS_DUE_DAYS_MAX','POS_DUE_DAYS_MEAN']

In [81]:
postemp.head()

SK_ID_CURR  POS_SK_ID_PREV_COUNT  POS_CREDIT_TERM_MEAN  POS_DUE_DAYS_COUNT  \
0      100001                     2              4.000000                   1   
1      100002                     1             24.000000                   0   
2      100003                     3             10.107143                   0   
3      100004                     1              3.750000                   0   
4      100005                     1             11.700000                   0   

   POS_DUE_DAYS_MAX  POS_DUE_DAYS_MEAN  
0                 7           0.777778  
1                 0           0.000000  
2                 0           0.000000  
3                 0           0.000000  
4                 0           0.000000

In [82]:
installments['daysoverdue'] = abs(installments['DAYS_ENTRY_PAYMENT'])-abs(installments['DAYS_INSTALMENT'])

In [83]:
instemp = installments.groupby('SK_ID_CURR', as_index = False).agg({'SK_ID_PREV':'nunique','NUM_INSTALMENT_NUMBER':'max',
                                                         'AMT_INSTALMENT':'sum','daysoverdue':['mean','max']})


instemp.columns = ['SK_ID_CURR','INS_SK_ID_PREV_COUNT','INS_NUM_MAX','INS_AMT_SUM','INS_DAYS_OVERDUE_MEAN','INS_DAYS_OVERDUE_MAX']

In [84]:
instemp.head()

SK_ID_CURR  INS_SK_ID_PREV_COUNT  INS_NUM_MAX  INS_AMT_SUM  \
0      100001                     2            4    41195.925   
1      100002                     1           19   219625.695   
2      100003                     3           12  1618864.650   
3      100004                     1            3    21288.465   
4      100005                     1            9    56161.845   

   INS_DAYS_OVERDUE_MEAN  INS_DAYS_OVERDUE_MAX  
0               7.285714                  36.0  
1              20.421053                  31.0  
2               7.160000                  14.0  
3               7.666667                  11.0  
4              23.555556                  37.0

In [85]:
credit.head()

SK_ID_PREV  SK_ID_CURR  MONTHS_BALANCE  AMT_BALANCE  \
0     2562384      378907              -6       56.970   
1     2582071      363914              -1    63975.555   
2     1740877      371185              -7    31815.225   
3     1389973      337855              -4   236572.110   
4     1891521      126868              -1   453919.455   

   AMT_CREDIT_LIMIT_ACTUAL  AMT_DRAWINGS_ATM_CURRENT  AMT_DRAWINGS_CURRENT  \
0                   135000                       0.0                 877.5   
1                    45000                    2250.0                2250.0   
2                   450000                       0.0                   0.0   
3                   225000                    2250.0                2250.0   
4                   450000                       0.0               11547.0   

   AMT_DRAWINGS_OTHER_CURRENT  AMT_DRAWINGS_POS_CURRENT  \
0                         0.0                     877.5   
1                         0.0                       0.0   
2                         0.0                       0.0   
3                         0.0                       0.0   
4                         0.0                   11547.0   

   AMT_INST_MIN_REGULARITY  AMT_PAYMENT_CURRENT  AMT_PAYMENT_TOTAL_CURRENT  \
0                 1700.325               1800.0                     1800.0   
1                 2250.000               2250.0                     2250.0   
2                 2250.000               2250.0                     2250.0   
3                11795.760              11925.0                    11925.0   
4                22924.890              27000.0                    27000.0   

   AMT_RECEIVABLE_PRINCIPAL  AMT_RECIVABLE  AMT_TOTAL_RECEIVABLE  \
0                     0.000          0.000                 0.000   
1                 60175.080      64875.555             64875.555   
2                 26926.425      31460.085             31460.085   
3                224949.285     233048.970            233048.970   
4                443044.395     453919.455            453919.455   

   CNT_DRAWINGS_ATM_CURRENT  CNT_DRAWINGS_CURRENT  CNT_DRAWINGS_OTHER_CURRENT  \
0                       0.0                     1                         0.0   
1                       1.0                     1                         0.0   
2                       0.0                     0                         0.0   
3                       1.0                     1                         0.0   
4                       0.0                     1                         0.0   

   CNT_DRAWINGS_POS_CURRENT  CNT_INSTALMENT_MATURE_CUM NAME_CONTRACT_STATUS  \
0                       1.0                       35.0               Active   
1                       0.0                       69.0               Active   
2                       0.0                       30.0               Active   
3                       0.0                       10.0               Active   
4                       1.0                      101.0               Active   

   SK_DPD  SK_DPD_DEF  exceedslimitby  exceedsflag  fullpaymentflag  \
0       0           0     -134943.030          0.0              1.0   
1       0           0       18975.555          1.0              0.0   
2       0           0     -418184.775          0.0              0.0   
3       0           0       11572.110          1.0              0.0   
4       0           0        3919.455          1.0              0.0   

   minpaymentflag  
0             1.0  
1             1.0  
2             1.0  
3             1.0  
4             1.0

In [86]:
credit['NAME_CONTRACT_STATUS'] = credit['NAME_CONTRACT_STATUS'].astype('category')
credit['NAME_CONTRACT_STATUS'].dtype

CategoricalDtype(categories=['Active', 'Approved', 'Completed', 'Demand', 'Refused',
                  'Sent proposal', 'Signed'],
                 ordered=False)

In [87]:
credit['exceedslimitby'] = credit['AMT_BALANCE'] - credit['AMT_CREDIT_LIMIT_ACTUAL']
credit.loc[credit['exceedslimitby'] >= 0,'exceedsflag'] = 1
credit.loc[credit['exceedslimitby'] < 0,'exceedsflag'] = 0

In [88]:
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_BALANCE'] >= 0,'fullpaymentflag'] = 1
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_BALANCE'] < 0,'fullpaymentflag'] = 0

In [89]:
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_INST_MIN_REGULARITY'] >= 0,'minpaymentflag'] = 1
credit.loc[credit['AMT_PAYMENT_TOTAL_CURRENT'] - credit['AMT_INST_MIN_REGULARITY'] < 0,'minpaymentflag'] = 0

In [90]:
crtemp1 = credit.groupby('SK_ID_CURR', as_index = False).agg({'AMT_BALANCE':'mean','AMT_CREDIT_LIMIT_ACTUAL':'mean',
'exceedslimitby':'mean','exceedsflag':'mean','fullpaymentflag':'mean','minpaymentflag':'mean'
,'AMT_DRAWINGS_CURRENT':'mean','CNT_DRAWINGS_CURRENT':'mean'})

In [91]:
crtemp1.head()

SK_ID_CURR   AMT_BALANCE  AMT_CREDIT_LIMIT_ACTUAL  exceedslimitby  \
0      100006      0.000000            270000.000000  -270000.000000   
1      100011  54482.111149            164189.189189  -109707.078041   
2      100013  18159.919219            131718.750000  -113558.830781   
3      100021      0.000000            675000.000000  -675000.000000   
4      100023      0.000000            135000.000000  -135000.000000   

   exceedsflag  fullpaymentflag  minpaymentflag  AMT_DRAWINGS_CURRENT  \
0     0.000000         1.000000        1.000000              0.000000   
1     0.040541         0.567568        0.986301           2432.432432   
2     0.010417         0.822917        0.921348           5953.125000   
3     0.000000         1.000000        1.000000              0.000000   
4     0.000000         1.000000        1.000000              0.000000   

   CNT_DRAWINGS_CURRENT  
0              0.000000  
1              0.054054  
2              0.239583  
3              0.000000  
4              0.000000

In [92]:
crtemp = pd.get_dummies(credit, columns = ['NAME_CONTRACT_STATUS'], prefix = 'CREDIT_CONT_STAT')
collist = list(crtemp.columns[crtemp.columns.str.startswith('CREDIT_CONT_STAT')])
collist.append('SK_ID_CURR')
collist.append('SK_ID_PREV')

In [93]:
credit['NAME_CONTRACT_STATUS'].value_counts()

Active           3698436
Completed         128918
Signed             11058
Demand              1365
Sent proposal        513
Refused               17
Approved               5
Name: NAME_CONTRACT_STATUS, dtype: int64

In [94]:
crtemp = crtemp.groupby(['SK_ID_CURR','SK_ID_PREV'], as_index = False).max()[collist]

In [95]:
crtemp.head()

CREDIT_CONT_STAT_Active  CREDIT_CONT_STAT_Approved  \
0                        1                          0   
1                        1                          0   
2                        1                          0   
3                        1                          0   
4                        1                          0   

   CREDIT_CONT_STAT_Completed  CREDIT_CONT_STAT_Demand  \
0                           0                        0   
1                           0                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   CREDIT_CONT_STAT_Refused  CREDIT_CONT_STAT_Sent proposal  \
0                         0                               0   
1                         0                               0   
2                         0                               0   
3                         0                               0   
4                         0                               0   

   CREDIT_CONT_STAT_Signed  SK_ID_CURR  SK_ID_PREV  
0                        0      100006     1489396  
1                        0      100011     1843384  
2                        0      100013     2038692  
3                        0      100021     2594025  
4                        0      100023     1499902

In [96]:
crtemp2 = crtemp.groupby('SK_ID_CURR', as_index = False).sum()
crtemp2.drop(columns = 'SK_ID_PREV', inplace = True )
crtemp2.head()

SK_ID_CURR  CREDIT_CONT_STAT_Active  CREDIT_CONT_STAT_Approved  \
0      100006                        1                          0   
1      100011                        1                          0   
2      100013                        1                          0   
3      100021                        1                          0   
4      100023                        1                          0   

   CREDIT_CONT_STAT_Completed  CREDIT_CONT_STAT_Demand  \
0                           0                        0   
1                           0                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   CREDIT_CONT_STAT_Refused  CREDIT_CONT_STAT_Sent proposal  \
0                         0                               0   
1                         0                               0   
2                         0                               0   
3                         0                               0   
4                         0                               0   

   CREDIT_CONT_STAT_Signed  
0                        0  
1                        0  
2                        0  
3                        0  
4                        0

In [97]:
credittemp = pd.merge(crtemp1,crtemp2, on = 'SK_ID_CURR', how = 'left')

In [98]:
credittemp.head()

SK_ID_CURR   AMT_BALANCE  AMT_CREDIT_LIMIT_ACTUAL  exceedslimitby  \
0      100006      0.000000            270000.000000  -270000.000000   
1      100011  54482.111149            164189.189189  -109707.078041   
2      100013  18159.919219            131718.750000  -113558.830781   
3      100021      0.000000            675000.000000  -675000.000000   
4      100023      0.000000            135000.000000  -135000.000000   

   exceedsflag  fullpaymentflag  minpaymentflag  AMT_DRAWINGS_CURRENT  \
0     0.000000         1.000000        1.000000              0.000000   
1     0.040541         0.567568        0.986301           2432.432432   
2     0.010417         0.822917        0.921348           5953.125000   
3     0.000000         1.000000        1.000000              0.000000   
4     0.000000         1.000000        1.000000              0.000000   

   CNT_DRAWINGS_CURRENT  CREDIT_CONT_STAT_Active  CREDIT_CONT_STAT_Approved  \
0              0.000000                        1                          0   
1              0.054054                        1                          0   
2              0.239583                        1                          0   
3              0.000000                        1                          0   
4              0.000000                        1                          0   

   CREDIT_CONT_STAT_Completed  CREDIT_CONT_STAT_Demand  \
0                           0                        0   
1                           0                        0   
2                           0                        0   
3                           1                        0   
4                           0                        0   

   CREDIT_CONT_STAT_Refused  CREDIT_CONT_STAT_Sent proposal  \
0                         0                               0   
1                         0                               0   
2                         0                               0   
3                         0                               0   
4                         0                               0   

   CREDIT_CONT_STAT_Signed  
0                        0  
1                        0  
2                        0  
3                        0  
4                        0

In [99]:
credittemp.shape

(103558, 16)

In [100]:
credittemp.columns = ['SK_ID_CURR','CR_AMT_BALANCE_MEAN','CR_CREDIT_LIMIT_MEAN','CR_EXCEED_LIMIT_AMT_MEAN',
                      'CR_EXCEED_LIMIT_CNT_MEAN','CR_FULL_PAYMENT_COUNT_MEAN','CR_MIN_PAYMENT_COUNT_MEAN',
                     'CR_DRAWINGS_AMT_MEAN','CR_DRAWINGS_CNT_MEAN','CR_CREDIT_CNT_ACTIVE','CR_CREDIT_CNT_APPROVED',
'CR_CREDIT_CNT_COMPLETED','CR_CREDIT_CNT_DEMAND','CR_CREDIT_CNT_REFUSED','CR_CREDIT_CNT_SENTPROPOSAL','CR_CREDIT_CNT_SIGNED']

In [101]:
credittemp.head()

SK_ID_CURR  CR_AMT_BALANCE_MEAN  CR_CREDIT_LIMIT_MEAN  \
0      100006             0.000000         270000.000000   
1      100011         54482.111149         164189.189189   
2      100013         18159.919219         131718.750000   
3      100021             0.000000         675000.000000   
4      100023             0.000000         135000.000000   

   CR_EXCEED_LIMIT_AMT_MEAN  CR_EXCEED_LIMIT_CNT_MEAN  \
0            -270000.000000                  0.000000   
1            -109707.078041                  0.040541   
2            -113558.830781                  0.010417   
3            -675000.000000                  0.000000   
4            -135000.000000                  0.000000   

   CR_FULL_PAYMENT_COUNT_MEAN  CR_MIN_PAYMENT_COUNT_MEAN  \
0                    1.000000                   1.000000   
1                    0.567568                   0.986301   
2                    0.822917                   0.921348   
3                    1.000000                   1.000000   
4                    1.000000                   1.000000   

   CR_DRAWINGS_AMT_MEAN  CR_DRAWINGS_CNT_MEAN  CR_CREDIT_CNT_ACTIVE  \
0              0.000000              0.000000                     1   
1           2432.432432              0.054054                     1   
2           5953.125000              0.239583                     1   
3              0.000000              0.000000                     1   
4              0.000000              0.000000                     1   

   CR_CREDIT_CNT_APPROVED  CR_CREDIT_CNT_COMPLETED  CR_CREDIT_CNT_DEMAND  \
0                       0                        0                     0   
1                       0                        0                     0   
2                       0                        0                     0   
3                       0                        1                     0   
4                       0                        0                     0   

   CR_CREDIT_CNT_REFUSED  CR_CREDIT_CNT_SENTPROPOSAL  CR_CREDIT_CNT_SIGNED  
0                      0                           0                     0  
1                      0                           0                     0  
2                      0                           0                     0  
3                      0                           0                     0  
4                      0                           0                     0

### Merging the engineered pos, installment and credit data set with engineered previous application data set to build the modified previous application data set included with all the newly derived features 

In [102]:
prevtemp = dfinal2

In [103]:
prevtemp.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [104]:
prevtemp.shape

(338857, 130)

In [105]:
prevtemp1 = pd.merge(prevtemp, credittemp, on = 'SK_ID_CURR', how = 'left')
prevtemp1.fillna(0, inplace = True)
prevtemp2 = pd.merge(prevtemp1, instemp, on = 'SK_ID_CURR', how = 'left')
prevtempfinal = pd.merge(prevtemp2,postemp,on = 'SK_ID_CURR', how = 'left')
#prevtempfinal.drop(columns = ['SK_ID_PREV_COUNT_x'], axis =0 , inplace = True)
prevtempfinal.fillna(0, inplace = True)

In [106]:
prevtempfinal.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [107]:
prevtempfinal.shape

(338857, 155)

In [108]:
prevtempfinal.columns

Index(['SK_ID_CURR', 'AMT_CREDIT', 'WEEKDAY_APPR_PROCESS_START',
       'HOUR_APPR_PROCESS_START', 'RATE_INTEREST_PRIMARY',
       'PREV_NCS_COUNT_Approved', 'PREV_NCS_COUNT_Canceled',
       'PREV_NCS_COUNT_Refused', 'PREV_NCS_COUNT_Unused offer',
       'PREV_NCLT_COUNT_New',
       ...
       'INS_SK_ID_PREV_COUNT_y', 'INS_NUM_MAX_y', 'INS_AMT_SUM_y',
       'INS_DAYS_OVERDUE_MEAN_y', 'INS_DAYS_OVERDUE_MAX_y',
       'POS_SK_ID_PREV_COUNT_y', 'POS_CREDIT_TERM_MEAN_y',
       'POS_DUE_DAYS_COUNT_y', 'POS_DUE_DAYS_MAX_y', 'POS_DUE_DAYS_MEAN_y'],
      dtype='object', length=155)

In [ ]:
#prevtempfinal.to_csv("prevtempfinal.csv")

In [109]:
dfinal2 = prevtempfinal

In [110]:
dfinal2.head()

SK_ID_CURR  AMT_CREDIT  WEEKDAY_APPR_PROCESS_START  \
0      100001    23787.00                         5.0   
1      100002   179055.00                         6.0   
2      100003   484191.00                         4.0   
3      100004    20106.00                         5.0   
4      100005    20076.75                         4.0   

   HOUR_APPR_PROCESS_START  RATE_INTEREST_PRIMARY  PREV_NCS_COUNT_Approved  \
0                13.000000                    0.0                        1   
1                 9.000000                    0.0                        1   
2                14.666667                    0.0                        3   
3                 5.000000                    0.0                        1   
4                10.500000                    0.0                        1   

   PREV_NCS_COUNT_Canceled  PREV_NCS_COUNT_Refused  \
0                        0                       0   
1                        0                       0   
2                        0                       0   
3                        0                       0   
4                        1                       0   

   PREV_NCS_COUNT_Unused offer  PREV_NCLT_COUNT_New  \
0                            0                  0.0   
1                            0                  1.0   
2                            0                  0.0   
3                            0                  1.0   
4                            0                  1.0   

   PREV_NCLT_COUNT_Refreshed  PREV_NCLT_COUNT_Repeater  PREV_NCLT_COUNT_XNA  \
0                        1.0                       0.0                  0.0   
1                        0.0                       0.0                  0.0   
2                        2.0                       1.0                  0.0   
3                        0.0                       0.0                  0.0   
4                        0.0                       1.0                  0.0   

   PREV_NCT_COUNT_Cash loans  PREV_NCT_COUNT_Consumer loans  \
0                        0.0                            1.0   
1                        0.0                            1.0   
2                        1.0                            2.0   
3                        0.0                            1.0   
4                        1.0                            1.0   

   PREV_NCT_COUNT_Revolving loans  PREV_NCT_COUNT_XNA  \
0                             0.0                 0.0   
1                             0.0                 0.0   
2                             0.0                 0.0   
3                             0.0                 0.0   
4                             0.0                 0.0   

   PREV_NCLP_COUNT_Building a house or an annex  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Business development  PREV_NCLP_COUNT_Buying a garage  \
0                                   0.0                              0.0   
1                                   0.0                              0.0   
2                                   0.0                              0.0   
3                                   0.0                              0.0   
4                                   0.0                              0.0   

   PREV_NCLP_COUNT_Buying a holiday home / land  \
0                                           0.0   
1                                           0.0   
2                                           0.0   
3                                           0.0   
4                                           0.0   

   PREV_NCLP_COUNT_Buying a home  PREV_NCLP_COUNT_Buying a new car  \
0                            0.0                               0.0   
1                            0.0                               0.0   
2                            0.0                         

In [111]:
dfinal2.shape

(338857, 155)

In [112]:
dfinal1.shape

(307511, 130)

## Merging updated app_train data set with updated previous application data set to build the final data set

In [113]:
dfinal = pd.merge(dfinal1,dfinal2, on = 'SK_ID_CURR', how = 'left')

In [114]:
dfinal.head()

SK_ID_CURR  TARGET NAME_CONTRACT_TYPE CODE_GENDER FLAG_OWN_CAR  \
0      100002       1         Cash loans           M            N   
1      100003       0         Cash loans           F            N   
2      100004       0    Revolving loans           M            Y   
3      100006       0         Cash loans           F            N   
4      100007       0         Cash loans           M            N   

  FLAG_OWN_REALTY  CNT_CHILDREN  AMT_INCOME_TOTAL  AMT_CREDIT_x  AMT_ANNUITY  \
0               Y             0          202500.0      406597.5      24700.5   
1               N             0          270000.0     1293502.5      35698.5   
2               Y             0           67500.0      135000.0       6750.0   
3               Y             0          135000.0      312682.5      29686.5   
4               Y             0          121500.0      513000.0      21865.5   

   AMT_GOODS_PRICE NAME_TYPE_SUITE NAME_INCOME_TYPE  \
0         351000.0   Unaccompanied          Working   
1        1129500.0          Family    State servant   
2         135000.0   Unaccompanied          Working   
3         297000.0   Unaccompanied          Working   
4         513000.0   Unaccompanied          Working   

             NAME_EDUCATION_TYPE    NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  \
0  Secondary / secondary special  Single / not married  House / apartment   
1               Higher education               Married  House / apartment   
2  Secondary / secondary special  Single / not married  House / apartment   
3  Secondary / secondary special        Civil marriage  House / apartment   
4  Secondary / secondary special  Single / not married  House / apartment   

   REGION_POPULATION_RELATIVE  DAYS_BIRTH  DAYS_EMPLOYED  DAYS_REGISTRATION  \
0                    0.018801       -9461           -637            -3648.0   
1                    0.003541      -16765          -1188            -1186.0   
2                    0.010032      -19046           -225            -4260.0   
3                    0.008019      -19005          -3039            -9833.0   
4                    0.028663      -19932          -3038            -4311.0   

   DAYS_ID_PUBLISH  OWN_CAR_AGE  FLAG_MOBIL  FLAG_EMP_PHONE  FLAG_WORK_PHONE  \
0            -2120          NaN           1               1                0   
1             -291          NaN           1               1                0   
2            -2531         26.0           1               1                1   
3            -2437          NaN           1               1                0   
4            -3458          NaN           1               1                0   

   FLAG_CONT_MOBILE  FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  CNT_FAM_MEMBERS  \
0                 1           1           0        Laborers              1.0   
1                 1           1           0      Core staff              2.0   
2                 1           1           0        Laborers              1.0   
3                 1           0           0        Laborers              2.0   
4                 1           0           0      Core staff              1.0   

   REGION_RATING_CLIENT  REGION_RATING_CLIENT_W_CITY  \
0                     2                            2   
1                     1                            1   
2                     2                            2   
3                     2                            2   
4                     2                            2   

  WEEKDAY_APPR_PROCESS_START_x  HOUR_APPR_PROCESS_START_x  \
0                    WEDNESDAY                         10   
1                       MONDAY                         11   
2                       MONDAY                          9   
3                    WEDNESDAY                         17   
4                     THURSDAY                         11   

   REG_REGION_NOT_LIVE_REGION  REG_REGION_NOT_WORK_REGION  \
0                           0                           0   
1                           0                     

In [115]:
dfinal.shape

(307511, 284)

## segregating categorical and numerical columns

In [116]:
cat_cols = ['NAME_CONTRACT_TYPE','NAME_TYPE_SUITE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY',
                       'CNT_CHILDREN','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS',
                       'NAME_HOUSING_TYPE','FLAG_MOBIL','FLAG_EMP_PHONE','FLAG_WORK_PHONE','FLAG_CONT_MOBILE','FLAG_PHONE',
                       'FLAG_EMAIL','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY',
                      'REG_REGION_NOT_LIVE_REGION','REG_REGION_NOT_WORK_REGION','LIVE_REGION_NOT_WORK_REGION',
                       'REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','LIVE_CITY_NOT_WORK_CITY','ORGANIZATION_TYPE','FLAG_DOCUMENT_2',
                       'FLAG_DOCUMENT_3','FLAG_DOCUMENT_4','FLAG_DOCUMENT_5','FLAG_DOCUMENT_6','FLAG_DOCUMENT_7','FLAG_DOCUMENT_8',
                       'FLAG_DOCUMENT_9','FLAG_DOCUMENT_10','FLAG_DOCUMENT_11','FLAG_DOCUMENT_12','FLAG_DOCUMENT_13',
                       'FLAG_DOCUMENT_14','FLAG_DOCUMENT_15','FLAG_DOCUMENT_16','FLAG_DOCUMENT_17','FLAG_DOCUMENT_18','FLAG_DOCUMENT_19',
                       'FLAG_DOCUMENT_20','FLAG_DOCUMENT_21','WALLSMATERIAL_MODE','HOUSETYPE_MODE',
            'FONDKAPREMONT_MODE','EMERGENCYSTATE_MODE','OCCUPATION_TYPE']

In [117]:
num_cols = set(dfinal.columns).difference(set(cat_cols))
num_cols = list(num_cols)
num_cols.remove('TARGET')
num_cols.remove('SK_ID_CURR')

In [118]:
dfinal[num_cols].info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Data columns (total 232 columns):
CR_AMT_BALANCE_MEAN_x                               float64
AMT_REQ_CREDIT_BUREAU_MON                           float64
FLOORSMAX_MEDI                                      float64
INS_SK_ID_PREV_COUNT_y                              float64
PREV_NTS_COUNT_Group of people                      float64
CR_CREDIT_CNT_SENTPROPOSAL_y                        float64
NONLIVINGAREA_MODE                                  float64
PREV_NGC_COUNT_Furniture                            float64
POS_CREDIT_TERM_MEAN_y                              float64
PREV_LOAN_COUNT                                     float64
INS_AMT_SUM_y                                       float64
NONLIVINGAPARTMENTS_AVG                             float64
INS_DAYS_OVERDUE_MAX_x                              float64
PREV_NCLT_COUNT_New                                 float64
CR_CREDIT_LIMIT_MEAN_y                          

In [119]:
dfinal[num_cols].isna().sum()

CR_AMT_BALANCE_MEAN_x                                16454
AMT_REQ_CREDIT_BUREAU_MON                            41519
FLOORSMAX_MEDI                                      153020
INS_SK_ID_PREV_COUNT_y                               16454
PREV_NTS_COUNT_Group of people                       16454
CR_CREDIT_CNT_SENTPROPOSAL_y                         16454
NONLIVINGAREA_MODE                                  169682
PREV_NGC_COUNT_Furniture                             16454
POS_CREDIT_TERM_MEAN_y                               16454
PREV_LOAN_COUNT                                      16454
INS_AMT_SUM_y                                        16454
NONLIVINGAPARTMENTS_AVG                             213514
INS_DAYS_OVERDUE_MAX_x                               16454
PREV_NCLT_COUNT_New                                  16454
CR_CREDIT_LIMIT_MEAN_y                               16454
EXT_SOURCE_1                                        173378
LANDAREA_MODE                                       1825

## Only considering numerical columns that have less that 45% of missing values

In [120]:
l = dfinal[num_cols].isna().sum() > dfinal.shape[0]*0.45
l = l[l == False]
num_cols = list(set(num_cols).intersection(set(l.index)))
num_cols

['CR_AMT_BALANCE_MEAN_x',
 'AMT_REQ_CREDIT_BUREAU_MON',
 'INS_SK_ID_PREV_COUNT_y',
 'PREV_NTS_COUNT_Group of people',
 'CR_CREDIT_CNT_SENTPROPOSAL_y',
 'CR_CREDIT_CNT_REFUSED_y',
 'PREV_NGC_COUNT_Furniture',
 'PREV_NTS_COUNT_Children',
 'BRU_AMT_MAX_OVERDUE',
 'POS_CREDIT_TERM_MEAN_y',
 'PREV_LOAN_COUNT',
 'PREV_NSI_COUNT_Consumer electronics',
 'INS_AMT_SUM_y',
 'HOUR_APPR_PROCESS_START_y',
 'INS_DAYS_OVERDUE_MAX_x',
 'POS_DUE_DAYS_COUNT_x',
 'PREV_NCLT_COUNT_New',
 'CR_CREDIT_LIMIT_MEAN_y',
 'REGION_POPULATION_RELATIVE',
 'PREV_CT_COUNT_Country-wide',
 'POS_DUE_DAYS_MEAN_y',
 'PREV_NCLP_COUNT_Wedding / gift / holiday',
 'POS_DUE_DAYS_COUNT_y',
 'CR_CREDIT_CNT_SENTPROPOSAL_x',
 'PREV_NTS_COUNT_Unaccompanied',
 'PREV_NGC_COUNT_Construction Materials',
 'INS_AMT_SUM_x',
 'AMT_ANNUITY',
 'PREV_NCLP_COUNT_Medicine',
 'PREV_NTS_COUNT_Spouse, partner',
 'PREV_CT_COUNT_Contact center',
 'CR_EXCEED_LIMIT_AMT_MEAN_y',
 'PREV_NSI_COUNT_Tourism',
 'WEEKDAY_APPR_PROCESS_START_y',
 'CR_CREDIT_LIMI

In [121]:
dfinal[num_cols].isna().sum()

CR_AMT_BALANCE_MEAN_x                               16454
AMT_REQ_CREDIT_BUREAU_MON                           41519
INS_SK_ID_PREV_COUNT_y                              16454
PREV_NTS_COUNT_Group of people                      16454
CR_CREDIT_CNT_SENTPROPOSAL_y                        16454
CR_CREDIT_CNT_REFUSED_y                             16454
PREV_NGC_COUNT_Furniture                            16454
PREV_NTS_COUNT_Children                             16454
BRU_AMT_MAX_OVERDUE                                 44020
POS_CREDIT_TERM_MEAN_y                              16454
PREV_LOAN_COUNT                                     16454
PREV_NSI_COUNT_Consumer electronics                 16454
INS_AMT_SUM_y                                       16454
HOUR_APPR_PROCESS_START_y                           16454
INS_DAYS_OVERDUE_MAX_x                              16454
POS_DUE_DAYS_COUNT_x                                16454
PREV_NCLT_COUNT_New                                 16454
CR_CREDIT_LIMI

In [122]:
dfinal[num_cols]['WEEKDAY_APPR_PROCESS_START_x'].head()

0    WEDNESDAY
1       MONDAY
2       MONDAY
3    WEDNESDAY
4     THURSDAY
Name: WEEKDAY_APPR_PROCESS_START_x, dtype: object

In [123]:
dfinal['WEEKDAY_APPR_PROCESS_START_x'] = dfinal['WEEKDAY_APPR_PROCESS_START_x'].map({'SUNDAY':0,'MONDAY': 1, 'TUESDAY':2, 'WEDNESDAY':3,'THURSDAY':4,
                                      'FRIDAY': 5, 'SATURDAY' : 6})

In [124]:
dfinal[num_cols].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Columns: 186 entries, CR_AMT_BALANCE_MEAN_x to PREV_NCT_COUNT_XNA
dtypes: float64(181), int64(5)
memory usage: 438.7 MB


## imputing N/A values with the respective means

In [125]:
for col in num_cols:
    dfinal[col] = dfinal[col].fillna(dfinal[col].mean())

## Only considering categorical columns that have less that 45% of missing values

In [126]:
c = dfinal[cat_cols].isna().sum() > dfinal.shape[0]*0.45
c = c[c == True]
cat_cols = list(set(cat_cols).difference(set(c.index)))

In [127]:
dfinal['NAME_TYPE_SUITE'] = dfinal['NAME_TYPE_SUITE'].fillna('Not Applicable')
dfinal['OCCUPATION_TYPE'] = dfinal['OCCUPATION_TYPE'].fillna('Not Applicable')

## converting categorical columns into dummies

In [128]:
dfinal[cat_cols]= dfinal[cat_cols].astype('category')
dfdum = pd.get_dummies(dfinal[cat_cols])
dfdum.shape

(307511, 204)

In [129]:
dfinal[cat_cols]= dfinal[cat_cols].astype('category')
dfdum = pd.get_dummies(dfinal[cat_cols])
dfdum.shape

(307511, 204)

In [130]:
dfinal[num_cols].shape

(307511, 186)

In [131]:
dfcat = pd.concat([dfinal[cat_cols], dfdum], axis = 1)
dfcat.drop(columns = cat_cols, inplace = True)
dfcat.shape

(307511, 204)

## concatenating numerical and categorical values to build the final data set having no missing values

In [132]:
dfet = pd.concat([dfinal[num_cols], dfcat], axis = 1)

In [133]:
dfet.shape

(307511, 390)

In [134]:
cat_cols = set(dfet.columns).difference(set(num_cols))
cat_cols = list(cat_cols)
for col in dfet[cat_cols].columns:
    dfet[col] = dfet[col].astype(np.int64)

In [135]:
dfet.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 307511 entries, 0 to 307510
Data columns (total 390 columns):
CR_AMT_BALANCE_MEAN_x                                float64
AMT_REQ_CREDIT_BUREAU_MON                            float64
INS_SK_ID_PREV_COUNT_y                               float64
PREV_NTS_COUNT_Group of people                       float64
CR_CREDIT_CNT_SENTPROPOSAL_y                         float64
CR_CREDIT_CNT_REFUSED_y                              float64
PREV_NGC_COUNT_Furniture                             float64
PREV_NTS_COUNT_Children                              float64
BRU_AMT_MAX_OVERDUE                                  float64
POS_CREDIT_TERM_MEAN_y                               float64
PREV_LOAN_COUNT                                      float64
PREV_NSI_COUNT_Consumer electronics                  float64
INS_AMT_SUM_y                                        float64
HOUR_APPR_PROCESS_START_y                            float64
INS_DAYS_OVERDUE_MAX_x            

In [136]:
dfet.columns

Index(['CR_AMT_BALANCE_MEAN_x', 'AMT_REQ_CREDIT_BUREAU_MON',
       'INS_SK_ID_PREV_COUNT_y', 'PREV_NTS_COUNT_Group of people',
       'CR_CREDIT_CNT_SENTPROPOSAL_y', 'CR_CREDIT_CNT_REFUSED_y',
       'PREV_NGC_COUNT_Furniture', 'PREV_NTS_COUNT_Children',
       'BRU_AMT_MAX_OVERDUE', 'POS_CREDIT_TERM_MEAN_y',
       ...
       'NAME_HOUSING_TYPE_Co-op apartment',
       'NAME_HOUSING_TYPE_House / apartment',
       'NAME_HOUSING_TYPE_Municipal apartment',
       'NAME_HOUSING_TYPE_Office apartment',
       'NAME_HOUSING_TYPE_Rented apartment', 'NAME_HOUSING_TYPE_With parents',
       'FLAG_DOCUMENT_4_0', 'FLAG_DOCUMENT_4_1', 'FLAG_DOCUMENT_13_0',
       'FLAG_DOCUMENT_13_1'],
      dtype='object', length=390)

In [137]:
dfet.head()

CR_AMT_BALANCE_MEAN_x  AMT_REQ_CREDIT_BUREAU_MON  INS_SK_ID_PREV_COUNT_y  \
0                    0.0                   0.000000                     1.0   
1                    0.0                   0.000000                     3.0   
2                    0.0                   0.000000                     1.0   
3                    0.0                   0.267395                     3.0   
4                    0.0                   0.000000                     5.0   

   PREV_NTS_COUNT_Group of people  CR_CREDIT_CNT_SENTPROPOSAL_y  \
0                             0.0                           0.0   
1                             0.0                           0.0   
2                             0.0                           0.0   
3                             0.0                           0.0   
4                             0.0                           0.0   

   CR_CREDIT_CNT_REFUSED_y  PREV_NGC_COUNT_Furniture  PREV_NTS_COUNT_Children  \
0                      0.0                       0.0                      0.0   
1                      0.0                       1.0                      0.0   
2                      0.0                       0.0                      0.0   
3                      0.0                       0.0                      0.0   
4                      0.0                       0.0                      0.0   

   BRU_AMT_MAX_OVERDUE  POS_CREDIT_TERM_MEAN_y  PREV_LOAN_COUNT  \
0          5043.645000               24.000000              1.0   
1             0.000000               10.107143              3.0   
2             0.000000                3.750000              1.0   
3          6557.470026               12.000000              9.0   
4             0.000000               15.333333              6.0   

   PREV_NSI_COUNT_Consumer electronics  INS_AMT_SUM_y  \
0                                  0.0     219625.695   
1                                  1.0    1618864.650   
2                                  0.0      21288.465   
3                                  1.0    1007153.415   
4                                  3.0     835985.340   

   HOUR_APPR_PROCESS_START_y  INS_DAYS_OVERDUE_MAX_x  POS_DUE_DAYS_COUNT_x  \
0                   9.000000                    31.0                   0.0   
1                  14.666667                    14.0                   0.0   
2                   5.000000                    11.0                   0.0   
3                  14.666667                    77.0                   0.0   
4                  12.333333                    31.0                   0.0   

   PREV_NCLT_COUNT_New  CR_CREDIT_LIMIT_MEAN_y  REGION_POPULATION_RELATIVE  \
0                  1.0                     0.0                    0.018801   
1                  0.0                     0.0                    0.003541   
2                  1.0                     0.0                    0.010032   
3                  1.0                270000.0                    0.008019   
4                  1.0                     0.0                    0.028663   

   PREV_CT_COUNT_Country-wide  POS_DUE_DAYS_MEAN_y  \
0                         0.0                  0.0   
1                         1.0                  0.0   
2                         0.0                  0.0   
3                         1.0                  0.0   
4                         3.0                  0.0   

   PREV_NCLP_COUNT_Wedding / gift / holiday  POS_DUE_DAYS_COUNT_y  \
0                                       0.0                   0.0   
1                                       0.0                   0.0   
2                                       0.0                   0.0   
3                                       0.0                   0.0   
4                                       0.0                   0.0   

   CR_CREDIT_CNT_SENTPROPOSAL_x  PREV_NTS_COUNT_Unaccompanied  \
0                           0.0                           0.0   
1                           0.0                           1.0   
2                  

In [139]:
dfet.to_csv('dfeature.csv', index = False)

In [140]:
print("The final data set is now ready")

The final data set is now ready


In [141]:
end_time = time.time()

In [142]:
timetaken = end_time - start_time
print("The total time taken to build the final data set is --- %s seconds ---" % (timetaken))

The total time taken to build the final data set is --- 1376.6989800930023 seconds ---


<b> Our final data set is now ready to be fed to the machine learning models and for further evaluations. In the next step I will run Logistic Regreesion, Random Forest, Light GBM and XGBoost Models on this data set and evaluate the respective results.

Please refer to the Home Credit Machine Learning Notebook to see how the models were run and what were the respective results. </b>